Imports

In [10]:
import pickle

import pandas as pd
import numpy as np

import json
from random import randint
from time import sleep

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from sklearn import cluster

Read files

In [53]:
spotify = pd.read_csv('spotify.csv')
top_lists = pd.read_csv('top_lists.csv')

In [45]:
std = pickle.load(open('std.p', 'rb'))
kmeans = pickle.load(open('kmeans.p', 'rb'))
# pred_ward = pickle.load(open('pred_ward.p', 'rb')) #These won't work ATM so i'll keep on working just with kmeans
# pred_complete = pickle.load(open('pred_complete.p', 'rb'))
# pred_average = pickle.load(open('pred_average.p', 'rb'))
# dbscan_opt = pickle.load(open('dbscan_opt.p', 'rb'))

Functions

In [20]:
def song_uri(song):
    q='track:'+ song# +'&artist:'+ artists
    uri = sp.search(q=q, limit=1)['tracks']['items'][0]['uri']

    return uri

def find_features(uri):
    try:
        features = sp.audio_features(uri)
    except:
        features = 0
    return features

def is_in_top(top_lists=top_lists):
    print('Please, write your song')
    name = input()

    songs = pd.Series(top_lists.song)

    flag = name in songs.unique()

    return flag, name

def top_reccomendation(name, top_lists=top_lists):
    l = len(top_lists)
    i = randint(0, l)
    while top_lists['song'][i] == name:
        i = randint(0, l)

    song = top_lists['song'][i]
    artist = top_lists['artist'][i]

    return song, artist

def recomend_v1(top_lists=top_lists):
    flag,name =  is_in_top(top_lists)

    if flag:
        song, artist = top_reccomendation(name)
        print('You can listen to ' + song + ' by ' + artist)
    else:
        print('Your song is not in the database')



Initialize

In [13]:
secrets_file = open("SpotifySecret.txt","r")
string = secrets_file.read()
secret_string = string.split('\n')

secrets_dict={}
for line in secret_string:
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['cs']))

New MVP

In [73]:
def recommender(spotify=spotify, tops=top_lists, std=std, kmeans=kmeans):
    flag,name =  is_in_top(top_lists)

    if flag:
        #If song in top list, returns another one
        song, artist = top_reccomendation(name)

    else:
        # Clustering
        uri = song_uri(name) #Needs treatment if not found
        features = pd.DataFrame(find_features(uri))
        features.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1, inplace=True)
        cols = features.columns
        features_t = std.transform(features)
        features_std = pd.DataFrame(features_t,columns=cols)

        cluster = kmeans.predict(features_std)[0]
        sublist = spotify.loc[spotify['kmeans_cluster'] == cluster][['song', 'artist']].reset_index(drop=True)
        i = randint(0, len(sublist)-1)

        song = sublist['song'][i]
        artist = sublist['artist'][i]

    print('You can listen to "' + song + '" by ' + artist)

        
        



In [74]:
recommender()

Please, write your song
You can listen to "Chop Suey!" by ['System Of A Down']


C:\Users\carlo\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


In [55]:
sublist = spotify.loc[spotify['kmeans_cluster'] == 6][['song', 'artist']]
i = randint(0, len(sublist)-1)
    # while sublist['song'][i] == 'name':
    #     i = randint(0, len(sublist)-1)

,song,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,type,id,uri.1,track_href,analysis_url,duration_ms,time_signature,kmeans_cluster
0,Crowd Chant,['Joe Satriani'],spotify:track:0bz67HYKfiuUj1xhsK5ofT,0.571,0.946,1,-6.830,0,0.0725,0.000203,...,0.2910,119.853,audio_features,0bz67HYKfiuUj1xhsK5ofT,spotify:track:0bz67HYKfiuUj1xhsK5ofT,https://api.spotify.com/v1/tracks/0bz67HYKfiuU...,https://api.spotify.com/v1/audio-analysis/0bz6...,194258,4,6
5,Ice,['Jesper Kyd'],spotify:track:7LJxBzIveF5uyjBgfT4UgU,0.756,0.336,1,-20.522,1,0.0368,0.369000,...,0.1250,98.999,audio_features,7LJxBzIveF5uyjBgfT4UgU,spotify:track:7LJxBzIveF5uyjBgfT4UgU,https://api.spotify.com/v1/tracks/7LJxBzIveF5u...,https://api.spotify.com/v1/audio-analysis/7LJx...,208733,4,6
6,Interlude Combat,['Jesper Kyd'],spotify:track:3u7449kz9nnlNlhx0j8GaC,0.588,0.497,2,-15.815,1,0.0379,0.001530,...,0.0718,125.008,audio_features,3u7449kz9nnlNlhx0j8GaC,spotify:track:3u7449kz9nnlNlhx0j8GaC,https://api.spotify.com/v1/tracks/3u7449kz9nnl...,https://api.spotify.com/v1/audio-analysis/3u74...,120053,4,6
9,Battlefield,"['Czech Philharmonic Chamber Orchestra', 'Kueh...",spotify:track:28RO5oP23oQ2Jxud6iby7q,0.405,0.792,7,-7.725,0,0.1580,0.642000,...,0.1570,99.081,audio_features,28RO5oP23oQ2Jxud6iby7q,spotify:track:28RO5oP23oQ2Jxud6iby7q,https://api.spotify.com/v1/tracks/28RO5oP23oQ2...,https://api.spotify.com/v1/audio-analysis/28RO...,401392,3,6
10,Hell March,['Frank Klepacki'],spotify:track:6Yyt7aFBMAxp0Eqzb1XGtl,0.575,0.810,2,-9.060,1,0.0499,0.003320,...,0.0674,121.437,audio_features,6Yyt7aFBMAxp0Eqzb1XGtl,spotify:track:6Yyt7aFBMAxp0Eqzb1XGtl,https://api.spotify.com/v1/tracks/6Yyt7aFBMAxp...,https://api.spotify.com/v1/audio-analysis/6Yyt...,384200,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11712,Demons,['DROELOE'],spotify:track:4WlSrA0dALD8NXSOTNhjNT,0.654,0.804,9,-5.437,1,0.0505,0.004310,...,0.2080,173.972,audio_features,4WlSrA0dALD8NXSOTNhjNT,spotify:track:4WlSrA0dALD8NXSOTNhjNT,https://api.spotify.com/v1/tracks/4WlSrA0dALD8...,https://api.spotify.com/v1/audio-analysis/4WlS...,284444,1,6
11734,Golden Empire feat. Zitaa - Original Mix,['Apashe'],spotify:track:1dX5seQ8Ygt3uEce7BreHH,0.665,0.650,10,-6.782,1,0.0335,0.056400,...,0.3000,100.014,audio_features,1dX5seQ8Ygt3uEce7BreHH,spotify:track:1dX5seQ8Ygt3uEce7BreHH,https://api.spotify.com/v1/tracks/1dX5seQ8Ygt3...,https://api.spotify.com/v1/audio-analysis/1dX5...,360442,4,6
11738,More (Prelude),['Apashe'],spotify:track:5euZjHtfURez3oTpUdesV0,0.257,0.523,2,-4.986,0,0.0328,0.040900,...,0.0371,72.807,audio_features,5euZjHtfURez3oTpUdesV0,spotify:track:5euZjHtfURez3oTpUdesV0,https://api.spotify.com/v1/tracks/5euZjHtfURez...,https://api.spotify.com/v1/audio-analysis/5euZ...,134400,4,6
11743,Kung Fu,"['Apashe', 'Dabin', 'Kai Wachi']",spotify:track:65mFyAs3kM32EGvcW2Pj4H,0.722,0.881,4,-5.013,0,0.0681,0.006600,...,0.1850,99.997,audio_features,65mFyAs3kM32EGvcW2Pj4H,spotify:track:65mFyAs3kM32EGvcW2Pj4H,https://api.spotify.com/v1/tracks/65mFyAs3kM32...,https://api.spotify.com/v1/audio-analysis/65mF...,244800,4,6


In [70]:
sublist = spotify.loc[spotify['kmeans_cluster'] == 6][['song', 'artist']].reset_index(drop=True)
i = randint(0, len(sublist)-1)
song = sublist['song'][i]
song

'Seven Nation Army'